In [1]:
import yfinance as yf
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as date
import bt
import PyQt5
import matplotlib

C:\Users\User\anaconda3\lib\site-packages\ffn\core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [4]:
%matplotlib qt4

ImportError: backend Gtk3Agg requires cairo

In [17]:
acoes = ["BIDI4.SA","LREN3.SA","ENBR3.SA","FLRY3.SA","WEGE3.SA","ABEV3.SA","GRND3.SA","ITSA4.SA","RADL3.SA"]
retornos = web.get_data_yahoo(acoes, start = date.datetime(2013,1,1))["Adj Close"].dropna().abs()
retornos.columns = retornos.columns.str.rstrip(".SA")

In [18]:
pct = retornos.pct_change()
pct.iloc[0,:] = 0

In [19]:
tx_livre = 0.02
cova = pd.DataFrame.cov(pct)*252
print(cova)
medias = np.mean(retornos.pct_change())*252
print(medias)
inver = np.linalg.inv(cova)
ret = medias-tx_livre
pesos_i = np.matmul(inver,ret)/np.sum(np.matmul(inver,ret))
pesos_i

Symbols     BIDI4     LREN3     ENBR3     FLRY3     WEGE3     ABEV3     GRND3  \
Symbols                                                                         
BIDI4    0.827241  0.118847  0.017003  0.105066  0.110562  0.079013  0.088029   
LREN3    0.118847  0.223919  0.081543  0.104013  0.086576  0.086098  0.092551   
ENBR3    0.017003  0.081543  0.120632  0.065345  0.057103  0.054022  0.058161   
FLRY3    0.105066  0.104013  0.065345  0.141271  0.072961  0.059872  0.068321   
WEGE3    0.110562  0.086576  0.057103  0.072961  0.177550  0.065635  0.061008   
ABEV3    0.079013  0.086098  0.054022  0.059872  0.065635  0.124293  0.055219   
GRND3    0.088029  0.092551  0.058161  0.068321  0.061008  0.055219  0.131121   
ITSA4    0.084516  0.110581  0.069967  0.070528  0.064901  0.067969  0.068242   
RADL3    0.037917  0.069037  0.046999  0.055553  0.066884  0.052006  0.047907   

Symbols     ITSA4     RADL3  
Symbols                      
BIDI4    0.084516  0.037917  
LREN3    0.110581 

array([ 0.39112248,  0.92445863,  1.09318966, -0.96192096,  2.47712204,
       -2.25333548, -0.74979994, -0.94048415,  1.01964772])

In [20]:
retornos

Symbols,BIDI4,LREN3,ENBR3,FLRY3,WEGE3,ABEV3,GRND3,ITSA4,RADL3
Date,,,,,,,,,
2018-05-02,71.678207,28.159914,12.488770,23.790468,17.392384,21.459694,8.016198,10.304420,12.826604
2018-05-03,72.589165,28.256294,12.277254,23.605904,16.996441,21.393433,7.890337,10.022644,12.706114
2018-05-04,68.216560,29.789572,12.102522,24.537958,16.610161,21.298771,7.615535,9.861653,12.243578
2018-05-07,69.712440,29.684423,12.258861,24.242655,16.339758,21.147314,7.507388,9.877683,11.924856
2018-05-08,71.054901,29.447815,12.185288,24.454903,16.281816,21.241974,7.586974,9.901858,12.111425
...,...,...,...,...,...,...,...,...,...
2020-11-13,21.059999,45.380001,18.990000,27.219999,82.639999,15.050000,8.100000,10.640000,26.639999
2020-11-16,21.719999,47.500000,18.969999,27.549999,81.580002,15.360000,8.380000,10.880000,27.030001
2020-11-17,21.420000,48.849998,18.990000,27.459999,81.690002,15.460000,8.350000,10.910000,26.469999


In [21]:
reb_m = bt.Strategy("Rebalanceamento Mensal",
                 [bt.algos.RunMonthly(run_on_end_of_period = True),
                  bt.algos.SelectAll(),
                  bt.algos.WeighSpecified(ABEV3 = 0.12595342,
                                          BIDI4 = 0.04690119,ENBR3 = 0.09624791,
                                          FLRY3 = 0.10516212,GRND3 = 0.07759859,
                                          ITSA4 = 0.11916407,LREN3 = 0.20068748,
                                          RADL3 = 0.05528098,WEGE3 = 0.17300424),
                  bt.algos.Rebalance()])
buy = bt.Strategy("Buy&Hold",
                 [bt.algos.RunOnce(),
                  bt.algos.SelectAll(),
                  bt.algos.WeighSpecified(ABEV3 = 0.12595342,
                                          BIDI4 = 0.04690119,ENBR3 = 0.09624791,
                                          FLRY3 = 0.10516212,GRND3 = 0.07759859,
                                          ITSA4 = 0.11916407,LREN3 = 0.20068748,
                                          RADL3 = 0.05528098,WEGE3 = 0.17300424),
                  bt.algos.Rebalance()])
reb_w = bt.Strategy("Rebalanceamento Semanal",
                 [bt.algos.RunWeekly(run_on_end_of_period = True),
                  bt.algos.SelectAll(),
                  bt.algos.WeighSpecified(ABEV3 = 0.12595342,
                                          BIDI4 = 0.04690119,ENBR3 = 0.09624791,
                                          FLRY3 = 0.10516212,GRND3 = 0.07759859,
                                          ITSA4 = 0.11916407,LREN3 = 0.20068748,
                                          RADL3 = 0.05528098,WEGE3 = 0.17300424),
                  bt.algos.Rebalance()])
reb_a = bt.Strategy("Rebalanceamento Anual",
                 [bt.algos.RunYearly(run_on_end_of_period = True),
                  bt.algos.SelectAll(),
                  bt.algos.WeighSpecified(ABEV3 = 0.12595342,
                                          BIDI4 = 0.04690119,ENBR3 = 0.09624791,
                                          FLRY3 = 0.10516212,GRND3 = 0.07759859,
                                          ITSA4 = 0.11916407,LREN3 = 0.20068748,
                                          RADL3 = 0.05528098,WEGE3 = 0.17300424),
                  bt.algos.Rebalance()])

In [22]:
bt1 = bt.Backtest(reb_m, retornos)
bt2 = bt.Backtest(buy, retornos)
bt3 = bt.Backtest(reb_w, retornos)
bt4 = bt.Backtest(reb_a, retornos)

In [23]:
result = bt.run(bt1, bt2, bt3, bt4)

Rebalanceamento Mensal
0% [############################# ] 100% | ETA: 00:00:00Buy&Hold


C:\Users\User\anaconda3\lib\site-packages\ffn\core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)


0% [############################# ] 100% | ETA: 00:00:00Rebalanceamento Semanal
0% [############################# ] 100% | ETA: 00:00:00Rebalanceamento Anual
0% [############################# ] 100% | ETA: 00:00:00

In [24]:
result.display()

Stat                 Rebalanceamento Mensal    Buy&Hold    Rebalanceamento Semanal    Rebalanceamento Anual
-------------------  ------------------------  ----------  -------------------------  -----------------------
Start                2018-05-01                2018-05-01  2018-05-01                 2018-05-01
End                  2020-11-19                2020-11-19  2020-11-19                 2020-11-19
Risk-free rate       0.00%                     0.00%       0.00%                      0.00%

Total Return         74.80%                    82.10%      74.57%                     77.28%
Daily Sharpe         0.90                      0.94        0.90                       0.91
Daily Sortino        1.32                      1.38        1.32                       1.34
CAGR                 24.44%                    26.45%      24.37%                     25.12%
Max Drawdown         -38.55%                   -38.25%     -38.68%                    -38.92%
Calmar Ratio         0.63        

In [46]:
result.plot();plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
result.get_transactions().tail()

price  quantity
Date       Security                     
2020-10-30 GRND3      7.996794    -423.0
           ITSA4      9.090000    -322.0
           LREN3     37.450001     582.0
           RADL3     24.059999     -37.0
           WEGE3     75.830002    -473.0

In [27]:
#Pesos Rebalanceamento mensal
result.get_security_weights()

,ABEV3,BIDI4,ENBR3,FLRY3,GRND3,ITSA4,LREN3,RADL3,WEGE3
2018-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-05-02,0.125947,0.046878,0.096238,0.105154,0.077597,0.119160,0.200668,0.055270,0.173002
2018-05-03,0.126889,0.047976,0.095611,0.105444,0.077188,0.117130,0.203488,0.055331,0.170855
2018-05-04,0.126337,0.045090,0.094257,0.109615,0.074505,0.115257,0.214546,0.053321,0.166985
2018-05-07,0.126163,0.046345,0.096027,0.108922,0.073872,0.116112,0.215024,0.052233,0.165215
...,...,...,...,...,...,...,...,...,...
2020-11-13,0.136673,0.052568,0.094410,0.091904,0.069026,0.122493,0.213553,0.053747,0.165544
2020-11-16,0.136851,0.053191,0.092528,0.091260,0.070063,0.122889,0.219304,0.053503,0.160332
2020-11-17,0.137005,0.052175,0.092129,0.090475,0.069438,0.122568,0.224329,0.052114,0.159688
2020-11-18,0.134194,0.053737,0.093267,0.090875,0.070261,0.122661,0.220800,0.052374,0.161750


In [28]:
#Pesos Buy&Hold
result.get_security_weights(1)

,ABEV3,BIDI4,ENBR3,FLRY3,GRND3,ITSA4,LREN3,RADL3,WEGE3
2018-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-05-02,0.125947,0.046878,0.096238,0.105154,0.077597,0.119160,0.200668,0.055270,0.173002
2018-05-03,0.126889,0.047976,0.095611,0.105444,0.077188,0.117130,0.203488,0.055331,0.170855
2018-05-04,0.126337,0.045090,0.094257,0.109615,0.074505,0.115257,0.214546,0.053321,0.166985
2018-05-07,0.126163,0.046345,0.096027,0.108922,0.073872,0.116112,0.215024,0.052233,0.165215
...,...,...,...,...,...,...,...,...,...
2020-11-13,0.048254,0.007524,0.079945,0.065727,0.042835,0.067218,0.176663,0.062711,0.449074
2020-11-16,0.048840,0.007696,0.079199,0.065973,0.043948,0.068165,0.183385,0.063103,0.439643
2020-11-17,0.048933,0.007555,0.078919,0.065456,0.043590,0.068039,0.187732,0.061512,0.438217
2020-11-18,0.047735,0.007749,0.079570,0.065480,0.043928,0.067816,0.184030,0.061568,0.442077


In [29]:
#Pesos Rebalanceamento anual
result.get_security_weights(2)

,ABEV3,BIDI4,ENBR3,FLRY3,GRND3,ITSA4,LREN3,RADL3,WEGE3
2018-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-05-02,0.125947,0.046878,0.096238,0.105154,0.077597,0.119160,0.200668,0.055270,0.173002
2018-05-03,0.126889,0.047976,0.095611,0.105444,0.077188,0.117130,0.203488,0.055331,0.170855
2018-05-04,0.125949,0.046882,0.096239,0.105151,0.077592,0.119154,0.200666,0.055276,0.172994
2018-05-07,0.125778,0.048188,0.098047,0.104488,0.076933,0.120040,0.201117,0.054149,0.171164
...,...,...,...,...,...,...,...,...,...
2020-11-13,0.125951,0.046892,0.096247,0.105161,0.077598,0.119164,0.200678,0.055277,0.172995
2020-11-16,0.126201,0.047480,0.094392,0.104495,0.078817,0.119630,0.206223,0.055064,0.167662
2020-11-17,0.126401,0.046595,0.094029,0.103644,0.078150,0.119373,0.211046,0.053659,0.167066
2020-11-18,0.123741,0.047964,0.095138,0.104046,0.079034,0.119399,0.207613,0.053897,0.169131


In [30]:
#Pesos Rebalanceamento semanal
result.get_security_weights(2)

,ABEV3,BIDI4,ENBR3,FLRY3,GRND3,ITSA4,LREN3,RADL3,WEGE3
2018-05-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-05-02,0.125947,0.046878,0.096238,0.105154,0.077597,0.119160,0.200668,0.055270,0.173002
2018-05-03,0.126889,0.047976,0.095611,0.105444,0.077188,0.117130,0.203488,0.055331,0.170855
2018-05-04,0.125949,0.046882,0.096239,0.105151,0.077592,0.119154,0.200666,0.055276,0.172994
2018-05-07,0.125778,0.048188,0.098047,0.104488,0.076933,0.120040,0.201117,0.054149,0.171164
...,...,...,...,...,...,...,...,...,...
2020-11-13,0.125951,0.046892,0.096247,0.105161,0.077598,0.119164,0.200678,0.055277,0.172995
2020-11-16,0.126201,0.047480,0.094392,0.104495,0.078817,0.119630,0.206223,0.055064,0.167662
2020-11-17,0.126401,0.046595,0.094029,0.103644,0.078150,0.119373,0.211046,0.053659,0.167066
2020-11-18,0.123741,0.047964,0.095138,0.104046,0.079034,0.119399,0.207613,0.053897,0.169131


In [48]:
result.plot_security_weights();plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
result.plot_histograms();plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …